### --- Day 11: Plutonian Pebbles ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/11

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 11);
WriteLines(inputLines);


1750884 193 866395 7 1158 31 35216 0


In [4]:
var inputLine = inputLines[0];

In [5]:
var testInputLine = "125 17";

In [6]:
using Stone = long; // int too small

LinkedList<Stone> ParseStones(string inputLine)
{
    var numbers = inputLine.Split(' ').Select(Stone.Parse);
    return new(numbers);
}

Let's start by modelling the stone operations:

In [7]:
delegate void StoneAction(LinkedList<Stone> stones, LinkedListNode<Stone> stone);

StoneAction AddOne = (stones, stone) => stone.Value = 1;
StoneAction Multiply2024 = (stones, stone) => stone.Value *= 2024;
StoneAction Split = (stones, stone) => {
    var stoneStr = stone.Value.ToString().AsSpan();
    var half = stoneStr.Length / 2;
    var firstHalf = Stone.Parse(stoneStr[0..half]);
    var secondHalf = Stone.Parse(stoneStr[half..]);

    stone.Value = firstHalf;
    stones.AddAfter(stone, secondHalf);
};

StoneAction Transform = (stones, stone) => {
    var func = stone.Value switch {
        // If the stone is engraved with the number 0, it is replaced by a stone
        // engraved with the number 1.
        0 => AddOne,
        
        // If the stone is engraved with a number that has an even number of
        // digits, it is replaced by two stones.
        var v when v.ToString().Length % 2 == 0 => Split,
        
        // If none of the other rules apply, the stone is replaced by a new
        // stone; the old stone's number multiplied by 2024 is engraved on the new
        // stone.
        _ => Multiply2024
    };
    func(stones, stone);
};


Now the blinking part is straightforward:

In [8]:
int BlinkAndCount(string inputLine, int times = 25)
{
    var stones = ParseStones(inputLine);
    foreach (var _ in Enumerable.Range(0, times))
    {
        BlinkOnce(stones);
    }
    return stones.Count;
}

void BlinkOnce(LinkedList<Stone> stones)
{
    foreach (var stone in stones.WalkNodes().ToList())
    {
        Transform(stones, stone);
    }
}

In [9]:
// In this example, after blinking six times, you would have 22 stones. After
// blinking 25 times, you would have 55312 stones!

var testAnswer = BlinkAndCount(testInputLine);
Console.WriteLine(testAnswer);

In [10]:
// Consider the arrangement of stones in front of you. How many stones will you
// have after blinking 25 times?

var part1Answer = BlinkAndCount(inputLine);
Console.WriteLine(part1Answer);

In [11]:
// 231278 is correct!
Ensure(231278, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/11

This naive approach takes waay too long, we'll need to think of something else:

In [13]:
// var part2Answer = BlinkAndCount(inputLine, times: 75);
// Console.WriteLine(part2Answer);

So looking at the breakdown of blink results from just the test string, it looks like we have a reasonable amount of repetition. 

Perhaps we can maintain a dictionary of stones -> counts, and increment the state of the dictionary on each blink.

In [14]:
// At 75 repetitions, some counts are too small for int!
using StoneCount = long;

In [15]:
using StoneDict = SCG.Dictionary<Stone, StoneCount>;

StoneDict ParseStones2(string inputLine)
{
    return inputLine.Split(' ').Select(Stone.Parse).GroupBy(st => st).ToDictionary(stGroup => stGroup.Key, stGroup => stGroup.LongCount());
}

Use a similar approach as part 1 i.e., defining transform functions, this time performing the transform `n` times for the `n` instances of the stone.

In [16]:
delegate void StoneDictAction(StoneDict dict, Stone stone, StoneCount nTimes);

void Replace(StoneDict dict, Stone oldStone, Stone newStone, StoneCount nTimes)
{
    dict[oldStone] -= nTimes;
    Increment(dict, newStone, nTimes);
}

StoneDictAction Increment = (dict, stone, nTimes) => {
    dict[stone] = dict.TryGetValue(stone, out var stoneCount) switch {
        true => stoneCount + nTimes,
        _ => nTimes
    };
};

StoneDictAction AddOne_2 = (dict, stone, nTimes) => Replace(dict, stone, stone + 1, nTimes);

StoneDictAction Multiply2024_2 = (dict, stone, times) => Replace(dict, stone, stone * 2024, times);

StoneDictAction Split_2 = (dict, stone, times) => {
    var stoneStr = stone.ToString().AsSpan();
    var half = stoneStr.Length / 2;
    var firstHalf = Stone.Parse(stoneStr[0..half]);
    var secondHalf = Stone.Parse(stoneStr[half..]);

    Replace(dict, stone, firstHalf, times);
    Increment(dict, secondHalf, times);
};

StoneDictAction Transform_2 = (dict, stone, times) => {
    var func = stone switch {
        0 => AddOne_2,
        var v when v.ToString().Length % 2 == 0 => Split_2,
        _ => Multiply2024_2
    };
    func(dict, stone, times);
};

In [17]:
long BlinkAndCount2(string inputLine, int times = 25)
{
    var stones = ParseStones2(inputLine);
    foreach (var _ in Enumerable.Range(0, times))
    {
        BlinkOnce2(stones);
    }

    return stones.Values.Sum();
}

void BlinkOnce2(StoneDict stones)
{
    foreach (var (stone, count) in stones.ToList())
    {
        if (count == 0) continue;

        Transform_2(stones, stone, count);
    }
}

In [18]:
// Ensure the provided sample still produces the same result for part 1, i.e.,
// 55312

var part2TestAnswer = BlinkAndCount2(testInputLine);
Console.WriteLine(part2TestAnswer);

In [19]:
// How many stones would you have after blinking a total of 75 times?

var part2Answer = BlinkAndCount2(inputLine, times: 75);
Console.WriteLine(part2Answer);

In [20]:
// 274229228071551 is correct!
Ensure(274229228071551, part2Answer);